In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
data = pd.read_csv("Data/sphist.csv")
print(data.head())

         Date         Open         High          Low        Close  \
0  2015-12-07  2090.419922  2090.419922  2066.780029  2077.070068   
1  2015-12-04  2051.239990  2093.840088  2051.239990  2091.689941   
2  2015-12-03  2080.709961  2085.000000  2042.349976  2049.620117   
3  2015-12-02  2101.709961  2104.270020  2077.110107  2079.510010   
4  2015-12-01  2082.929932  2103.370117  2082.929932  2102.629883   

         Volume    Adj Close  
0  4.043820e+09  2077.070068  
1  4.214910e+09  2091.689941  
2  4.306490e+09  2049.620117  
3  3.950640e+09  2079.510010  
4  3.712120e+09  2102.629883  


this dataset contains S&P500 stock opening, closing, high/low prices of 1950-2015

In [3]:
df = data.copy()
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values("Date", ascending = True, inplace = True)
print(df.head())

            Date   Open   High    Low  Close     Volume  Adj Close
16589 1950-01-03  16.66  16.66  16.66  16.66  1260000.0      16.66
16588 1950-01-04  16.85  16.85  16.85  16.85  1890000.0      16.85
16587 1950-01-05  16.93  16.93  16.93  16.93  2550000.0      16.93
16586 1950-01-06  16.98  16.98  16.98  16.98  2010000.0      16.98
16585 1950-01-09  17.08  17.08  17.08  17.08  2520000.0      17.08


In [5]:
i = 0
close_acc5 = 0
close_acc30 = 0
close_std5 = []
day5 = []
day30 = []
std5 = []

#calculation 
for idx, row in df.iterrows():
    i += 1
    if i <= 5:
        day5.append(0)
        day30.append(0)
        std5.append(0)
        close_acc5 += row["Close"]
        close_acc30 += row["Close"]
        close_std5.append(row["Close"])
    elif (i <= 30) & (i > 5):
        day5.append(close_acc5/5)
        day30.append(0)
        std5.append(np.std(close_std5))
        close_acc5 -= df.iloc[i-6,:]["Close"]
        close_acc5 += df.iloc[i-1,:]["Close"]
        close_std5.pop(0)
        close_std5.append(df.iloc[i-1,:]["Close"])
        close_acc30 += row["Close"]
    else:
        day30.append(close_acc30 / 30)
        close_acc30 -= df.iloc[i-31]["Close"]
        close_acc30 += df.iloc[i-1]["Close"]
        day5.append(close_acc5/5)
        std5.append(np.std(close_std5))
        close_std5.pop(0)
        close_std5.append(df.iloc[i-1,:]["Close"])
        close_acc5 -= df.iloc[i-6,:]["Close"]
        close_acc5 += df.iloc[i-1,:]["Close"]
df["day_5"] = day5
df["day_30"] = day30
df["std_5"] = std5
print(df.iloc[16330:16339])

[16.66, 16.85, 16.93, 16.98, 17.08]
[16.85, 16.93, 16.98, 17.08, 17.030001000000002]
[16.93, 16.98, 17.08, 17.030001000000002, 17.09]
[16.98, 17.08, 17.030001000000002, 17.09, 16.760000000000002]
[17.08, 17.030001000000002, 17.09, 16.760000000000002, 16.670000000000002]
[17.030001000000002, 17.09, 16.760000000000002, 16.670000000000002, 16.719998999999998]
[17.09, 16.760000000000002, 16.670000000000002, 16.719998999999998, 16.860001]
[16.760000000000002, 16.670000000000002, 16.719998999999998, 16.860001, 16.850000000000001]
[16.670000000000002, 16.719998999999998, 16.860001, 16.850000000000001, 16.870001000000002]
[16.719998999999998, 16.860001, 16.850000000000001, 16.870001000000002, 16.899999999999999]
[16.860001, 16.850000000000001, 16.870001000000002, 16.899999999999999, 16.920000000000002]
[16.850000000000001, 16.870001000000002, 16.899999999999999, 16.920000000000002, 16.860001]
[16.870001000000002, 16.899999999999999, 16.920000000000002, 16.860001, 16.739999999999998]
[16.899999

In [21]:
df = df[df["Date"] > datetime(year = 1950, month = 2, day = 14)]
train = df[df["Date"] < datetime(year = 2013, month = 1, day = 1)]
test = df[df["Date"] >= datetime(year = 2013, month = 1, day = 1)]
feature = ["day_5", "day_30", "std_5"]

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
target = "Close"
lr.fit(train[feature], train[target])
predict_close = lr.predict(test[feature])
mae = abs(predict_close - test[target]).mean()
print(train.tail())

          Date         Open         High          Low        Close  \
743 2012-12-24  1430.150024  1430.150024  1424.660034  1426.660034   
742 2012-12-26  1426.660034  1429.420044  1416.430054  1419.829956   
741 2012-12-27  1419.829956  1422.800049  1401.800049  1418.099976   
740 2012-12-28  1418.099976  1418.099976  1401.579956  1402.430054   
739 2012-12-31  1402.430054  1426.739990  1398.109985  1426.189941   

           Volume    Adj Close        day_5       day_30     std_5  
743  1.248960e+09  1426.660034  1437.360010  1405.926001  6.817332  
742  2.285030e+09  1419.829956  1436.620019  1407.486336  7.682855  
741  2.830180e+09  1418.099976  1431.228003  1408.813000  8.102333  
740  2.426680e+09  1402.430054  1427.685986  1410.265332  9.130821  
739  3.204330e+09  1426.189941  1419.434009  1411.830001  9.572036  


***additional indicator to consider***
- The average volume over the past five days.
- The average volume over the past year.
- The ratio between the average volume for the past five days, and the average volume for the past year.
- The standard deviation of the average volume over the past five days.
- The standard deviation of the average volume over the past year.
- The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
- The year component of the date.
- The ratio between the lowest price in the past year and the current price.
- The ratio between the highest price in the past year and the current price.
- The year component of the date.
- The month component of the date.
- The day of week.
- The day component of the date.
- The number of holidays in the prior month.